In [1]:
# standardized modules
import seaborn as sns
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.model_selection import train_test_split

#my modules
import acquire as ac
import prepare as pp
import helper

ModuleNotFoundError: No module named 'acquire'

In [6]:
helper.matrix()

## 2. Given the following confusion matrix, evaluate (by hand) the model's performance.


|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |

- In the context of this problem, what is a false positive?

cat is positive  
dog is negative

- In the context of this problem, what is a false negative?

Model predicted a cat. Actual image is of a dog.

- How would you describe this model?


In [13]:
TP = 34
FP = 7
FN = 13
TN = 46

# style taken from Adam's work
ALL = TP + FP + FN + TN

accuracy = (TP + TN)/ALL
true_positive_rate = sensitivity = recall = power = TP/(TP+FN)
false_positive_rate = false_alarm_ratio = fallout = FP/(FP+TN)
true_negative_rate = specificity = selectivity = TN/(TN+FP)
false_negative_rate = miss_rate = FN/(FN+TP)
precision = PPV = TP/(TP+FP)
f1_score = 2*(precision*recall)/(precision+recall)
support_pos = TP + FN
support_neg = FP + TN

print(f"Accuracy: {accuracy}", '\n')
print(f"True Positive Rate/Sensitivity/Recall/Power: {true_positive_rate}")
print(f"False Positive Rate/False Alarm Ratio/Fall-out: {false_positive_rate}")
print(f"True Negative Rate/Specificity/Selectivity: {true_negative_rate}")
print(f"False Negative Rate/Miss Rate: {false_negative_rate}")
print(f"Precision/PPV: {precision}")
print(f"F1 Score: {f1_score}")
print(f"Support (0): {support_neg}")
print(f"Support (1): {support_pos}")

Accuracy: 0.8 

True Positive Rate/Sensitivity/Recall/Power: 0.723404255319149
False Positive Rate/False Alarm Ratio/Fall-out: 0.1320754716981132
True Negative Rate/Specificity/Selectivity: 0.8679245283018868
False Negative Rate/Miss Rate: 0.2765957446808511
Precision/PPV: 0.8292682926829268
F1 Score: 0.7727272727272727
Support (0): 53
Support (1): 47


## 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

Use the predictions dataset and pandas to help answer the following questions:

- An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [15]:
duck = pd.read_csv('data/c3.csv')

In [16]:
duck.head()

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect


In [18]:
duck.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   actual  200 non-null    object
 1   model1  200 non-null    object
 2   model2  200 non-null    object
 3   model3  200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


In [20]:
duck.actual.value_counts()

No Defect    184
Defect        16
Name: actual, dtype: int64

Assigning to the positive defective ducks, making nondefective ducks negative:
We need to identify and maximize our model's sensitivity (recall).

recall = TP/(TP+FN)

In [22]:
sample = duck[duck.actual == 'Defect']
sample

,actual,model1,model2,model3
13,Defect,No Defect,Defect,Defect
30,Defect,Defect,No Defect,Defect
65,Defect,Defect,Defect,Defect
70,Defect,Defect,Defect,Defect
74,Defect,No Defect,No Defect,Defect
87,Defect,No Defect,Defect,Defect
118,Defect,No Defect,Defect,No Defect
135,Defect,Defect,No Defect,Defect
140,Defect,No Defect,Defect,Defect
147,Defect,Defect,No Defect,Defect


In [35]:
# Models used against a sample recall
for i in range(1,3):
    model_recall= (sample.actual == sample.columns[i]).mean()
    print(f'Model {i}: {model_recall:.2%}')

Model 1: 0.00%
Model 2: 0.00%


In [ ]:
# Model 2 recall


- Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you they really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. 

- Which evaluation metric would be appropriate here?

positive case = defect

## You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

- At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

- Several models have already been developed with the data, and you can find their results here.

- Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

In [5]:
paws = pd.read_csv('data/gives_you_paws.csv')

In [6]:
paws.head()

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog


In [7]:
paws.actual.value_counts()

dog    3254
cat    1746
Name: actual, dtype: int64

In [8]:
paws['baseline'] = paws.actual.value_counts().idxmax()

- In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

In [9]:
(paws.actual == paws.baseline).mean()

0.6508

In [18]:
# loop to go through accuracy of every model
model_acc = []

# model_recall= (sample.actual == sample.columns[i]).mean()
for i in paws.columns[1:]:
    acc = (paws.actual == paws[i]).mean()
    model_acc.append([i, acc])

model_acc

[['model1', 0.8074],
 ['model2', 0.6304],
 ['model3', 0.5096],
 ['model4', 0.7426],
 ['baseline', 0.6508]]

In [21]:
# make it purdy
pd.DataFrame(model_acc, columns=['i','accuracy'])

,i,accuracy
0,model1,0.8074
1,model2,0.6304
2,model3,0.5096
3,model4,0.7426
4,baseline,0.6508


Models 1 and 3 perform better than baseline